In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

# Import Tockler data

The input file is an Excel document that contains the following columns directly exported from Tockler: App, Type, Title, Begin, End.
It also contains an addition column called 'Activity' with the label of the project the entry corresponds to or with 'Interruption' if it corresponds to any other activity

In [ ]:
tockler_data = pd.read_excel("tockler_log.xlsx")
tockler_data['Begin'] = pd.to_datetime(tockler_data['Begin'].dt.tz_localize('Europe/Amsterdam'))
tockler_data['End'] = pd.to_datetime(tockler_data['End'].dt.tz_localize('Europe/Amsterdam'))
tockler_data['Source'] = 'Tockler'

In [ ]:
tockler_data

# Merge location data

We assume that folder_path points to a folder that contains the JSON files with the location information as exported from Google Maps

In [ ]:
def from_location_to_df(json_data):
    data= {
    "Source": [],
    "Begin": [],
    "End": [],
    "Activity": [],
    "Title": [],
    "Location": [],
    "Duration": [],
    }

    conv = []    

    ls_actividades=json_data['timelineObjects']
    for act in ls_actividades:
        if 'activitySegment' in act:
            valores=act['activitySegment']
            fecha_inicio=pd.to_datetime(valores['duration']['startTimestamp'])
            fecha_fin=pd.to_datetime(valores['duration']['endTimestamp'])
            ubicacion=valores['activityType']
        else:
            valores = act['placeVisit']
            fecha_inicio=pd.to_datetime(valores['duration']['startTimestamp'], utc=True)
            fecha_fin=pd.to_datetime(valores['duration']['endTimestamp'], utc=True)
            location = valores['location']
            if 'name' in location:
                ubicacion = location['name']
            else:
                ubicacion = location['address']

        conv.append({
            'Source':'Google Location', 
            'Begin':fecha_inicio,
            'End':fecha_fin, 
            'Activity':'',
            'Title':'', 
            'Location':ubicacion,
            'Duration':pd.to_datetime(fecha_fin)-pd.to_datetime(fecha_inicio)
        }) 

    return pd.DataFrame(conv)

import os
import json

folder_path = 'Location data'

all_locations = []
for f in os.listdir(folder_path):
     full_path = os.path.join(folder_path, f)
     if os.path.isfile(full_path):
          print(f"Processing {f}")
          with open(full_path) as file_open:
              json_data = json.load(file_open)

          all_locations.append(from_location_to_df(json_data))

df_location = pd.concat(all_locations).sort_values("Begin")

In [ ]:
merged = pd.concat([tockler_data, df_location]).sort_values("Begin").reset_index(drop=True)
merged

In [ ]:
locations_group = (~pd.isna(merged['Location'])).cumsum()
new_location = merged.groupby(locations_group)['Location'].transform('first')
location_start = merged.groupby(locations_group)['Begin'].transform('first')
location_end = merged.groupby(locations_group)['End'].transform('first')
apply_if = (location_start <= merged['Begin']) & (merged['Begin'] <= location_end)
merged.loc[apply_if, 'Location'] = new_location

In [ ]:
dto = merged[merged['Source'] == 'Tockler'].copy().reset_index(drop=True)
dto['Begin'] = pd.to_datetime(dto['Begin'])
dto['End'] = pd.to_datetime(dto['End'])
dto

In [ ]:
dto["Location"].value_counts()

In [ ]:
# We discard all locations whose only recorded windows are NATIVE / NO_TITLE because they are likely to be incorrectly recorded
discard_location = (dto["App"] == "NATIVE").groupby(dto["Location"]).all()
locations_discarded = discard_location[discard_location].index.values
locations_discarded

In [ ]:
dto = dto.drop(dto[dto["Location"].isin(locations_discarded)].index.values)

# Data preprocessing

In [ ]:
# We use the timeout to consider the inactivity time in the log
INACTIVITY_THRESHOLD = pd.Timedelta("1m")

# We consider it a change if the activity is different or it is a new day or if the gap between the end of an activity and the beginning of the next is greater than a threshold
change = (((dto["Activity"].shift() != dto["Activity"])) | (dto["Begin"].shift().dt.day != dto["End"].dt.day) | ((dto["Begin"] - dto["End"].shift()) > INACTIVITY_THRESHOLD))

In [ ]:
change

In [ ]:
sum(change)

In [ ]:
it = change.cumsum()

In [ ]:
it

In [ ]:
pr = dto.groupby(by=it).agg({"Begin": "first", "End": "last", "Activity": "first", "Title":"first", "Source":"count", "Location": "first"})
pr["Duration"] = pr["End"] - pr["Begin"]
pr["Begin"] = pd.to_datetime(pr["Begin"])
pr["End"] = pd.to_datetime(pr["End"])
pr["Duration_minutes"] = pr["Duration"] / pd.Timedelta('1m')
prev = pr["Activity"].shift()
prev.loc[pr["Begin"].dt.date != pr["Begin"].shift().dt.date] = np.nan
pr["Prev"] = prev
pr["OffComputer"] = ((pr["Begin"] - pr.shift()["End"] > INACTIVITY_THRESHOLD) & (pr["Begin"].dt.day == pr.shift()["End"].dt.day))

In [ ]:
# Now, each of the rows of pr represent a time slot of uninterrupted time either by electronic interruptions 
# that are included in the log or by physical or other types of interruptions, which are not included in the log
pr.head(50)

In [ ]:
# We keep in projs only those activities that relate to the projects under analysis
projs = pr[pr["Activity"] != "Interruption"].copy()

This is our definition of within-day interruption. It is the time between the end of a project activity and the beginning of a project activity that occurs within the same natural day. 

In [ ]:
interruption_time_b = projs.groupby("Activity").apply(lambda x: x["Begin"]- x["End"].shift(), include_groups=False)
interruption_time_b.loc[projs.groupby("Activity").apply(lambda x: x["Begin"].dt.day != x["End"].shift().dt.day, include_groups=False)] = np.nan
projs["interruption_time"] = interruption_time_b.reset_index(level=0, drop=True) / pd.Timedelta('1m')
projs.head(20)

And this is how we prepare a dataset that uses days as unit instead of activities. The reason behind this is that we do not consider an interruption the night of each day

In [ ]:
pdaily = projs.groupby(["Activity", projs["Begin"].dt.date]).agg(Duration=('Duration', "sum"), Times=('Duration', "count")).reset_index().sort_values(["Activity", "Begin"])
pdaily["Duration"] = pdaily["Duration"] / pd.Timedelta('1h')
pdaily["Times"] = pdaily["Times"] - 1
pdaily["Begin"] = pd.to_datetime(pdaily["Begin"]).dt.tz_localize('Europe/Amsterdam')
pdaily["Gap"] = pdaily.groupby('Activity')["Begin"].transform(lambda x: (x - x.shift())/pd.Timedelta('1d'))
pdaily.head(20)

In [ ]:
projs.groupby("Activity")["Begin"].agg(["first", "last"])

# Metrics

In [ ]:
def compute_metrics(freq = None, grouper = None, interruption_threshold=None):
    '''
    This function computes the metrics using the values of projs and pdaily.

    - freq refers to the temporal frequency used to compute the metrics (e.g. 1 week)
    - grouper refers to an optional attribute used to group the metrics
    - interruption_threshold refers to the threshold used to determine what an long interruption
    is (default = 60 minutes)

    '''
    groupby_spec = ["Activity"]
    if freq is not None:
        groupby_spec = groupby_spec + [pd.Grouper(key="Begin", freq=freq)]
    if grouper is not None:
        groupby_spec = groupby_spec + [grouper]
    if interruption_threshold is None:
        interruption_threshold = pd.Timedelta('60m') / pd.Timedelta('1m')

    g = projs.groupby(groupby_spec).agg(SlotDuration=("Duration_minutes", "mean"), InterruptionDuration=("interruption_time", "mean"), OffComputerInterruption=("OffComputer", "sum"))
    d = pdaily.groupby(groupby_spec).agg(TimesResumed=("Times", "sum"), TotalDurationHours=("Duration", "sum"), DaysGap=("Gap", "mean"))
    d["InterruptionsPerWorkHour"] = d["TimesResumed"] / d["TotalDurationHours"]
    if freq is not None:
        g2 = projs[projs["interruption_time"] < interruption_threshold].groupby(by=["Activity", pd.Grouper(key="Begin", freq=freq)])["interruption_time"].agg(ShortInterruptionDuration="mean", NumShortInterruption="count")
        g3 = projs[projs["interruption_time"] >= interruption_threshold].groupby(["Activity", pd.Grouper(key="Begin", freq=freq)])["interruption_time"].agg(LongInterruptionDuration="mean", NumLongInterruption="count")
        num_activities = pdaily.groupby(pd.Grouper(key="Begin", freq=freq))["Activity"].nunique().rename("ActivityVariety")
        return pd.merge(pd.concat([g, g2, g3, d], axis=1), num_activities, left_on="Begin", right_index=True, how="left")
    else:
        g2 = projs[projs["interruption_time"] < interruption_threshold].groupby(by="Activity")["interruption_time"].agg(ShortInterruptionDuration="mean", NumShortInterruption="count")
        g3 = projs[projs["interruption_time"] >= interruption_threshold].groupby("Activity")["interruption_time"].agg(LongInterruptionDuration="mean", NumLongInterruption="count")
        return pd.concat([g, g2, g3, d], axis=1)


In [ ]:
daily_metrics = compute_metrics(freq='D')
daily_metrics

In [ ]:
compute_metrics()

# Calendar

The calendar data is a CSV as exported by Microsoft Outlook in Windows

In [ ]:
calendar_csv = pd.read_csv("calendar_data.CSV", delimiter=',')
calendar_csv

# Location

In [ ]:
dto["Location"].value_counts()

## Trip dates

In [ ]:
travel = ["FLYING"]
df_location[((df_location["Location"].isin(travel))).shift(1, fill_value=False) | (df_location["Location"].str.contains("Airport"))]

In [ ]:
# This is a dict with the location and the country of the location

country = {
    "location1": "country1",
    "location2": "country1",
    "location3": "country2"
}    
home_country = "ZXXX"
abroad = [c for c in country if country[c]!=home_country]
travel = dto["Location"].isin(abroad)
location_change = dto.groupby((travel != travel.shift()).cumsum()).agg({"Begin": "first", "End":"last", "Location":"first"})
location_change[location_change["Location"].isin(abroad)]

In [ ]:
# This is a manual merge of the travels recognized as being flying or at the airport and the work abroad based on the output obtained from the previous cells

trip_dates = [
    pd.Timestamp(year=2023, month=1, day=2, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=1, day=26, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=3, day=25, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=4, day=28, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=5, day=23, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=6, day=10, tz="Europe/Amsterdam"),
    pd.Timestamp(year=2023, month=7, day=7, tz="Europe/Amsterdam")
]
    

## Mapping location to home / work / other

In [ ]:
# This is a map where you specify the location and whether this location should be considered Home / Work / Other

location_map = {
    "location1": "Home",
    "location2": "Work",
    "location3": "Home"
}

location_values = dto["Location"].dropna().unique()
location_mapped = dict()
for l in location_values:
    location_mapped[l] = location_map[l] if l in location_map else "Other"

location_mapped

In [ ]:
location_type = dto["Location"].replace(location_mapped).fillna("Other")

# Factors

In [ ]:
def compute_deadline (dataframe, deadline, activity=None, days=7):
    date_condition = (dataframe["Begin"] <= deadline) & (dataframe["Begin"] > (deadline - pd.Timedelta(days=days)))

    full_condition = date_condition & (dataframe["Activity"]==activity) if activity is not None else date_condition

    deadline_df = dataframe[full_condition]["Begin"]
    return days - (deadline - deadline_df) / pd.Timedelta(days=1)

def deadline_presure(dataframe, deadlines, activity=None, days=7):
    return pd.Series(pd.concat([compute_deadline(dataframe, d, activity=activity, days=days) for d in deadlines]) if len(deadlines) > 0 else 0.0, dataframe.index).fillna(0)


def compute_factor_deadline(dataframe, deadlines, days=7): 
    return pd.concat([deadline_presure(dataframe, deadlines[act], act, days=days) for act in deadlines], axis=1).sum(axis=1)

def compute_factor_others(dataframe, deadlines, days=7):
    deadline_df = pd.concat([deadline_presure(dataframe, deadlines[act], activity=None, days=days).rename(act + " deadline") for act in deadlines], axis=1)    
    return pd.concat([deadline_df.drop(act+" deadline", axis=1).sum(axis=1)[dataframe["Activity"]==act] for act in deadlines], axis=0)

def compute_factor_trip(dataframe, trip_dates, days=7):
    return deadline_presure(dataframe, deadlines=trip_dates, days=days)

def compute_factor_weekend_work(dataframe):
    return (dataframe["Begin"].dt.dayofweek >= 5) * 1.0

def compute_factor_summer_work(dataframe):
    return ((dataframe["Begin"].dt.month == 7) | ((dataframe["Begin"].dt.month == 8) & (dataframe["Begin"].dt.day < 15))) * 1.0

def compute_location(projs, location_mapped):
    loc_duration = projs["Duration"].groupby([projs["Activity"],  projs["Begin"].dt.date, projs["Location"].replace(location_mapped).fillna("Other")]).sum()
    all_duration = projs["Duration"].groupby([projs["Activity"],  projs["Begin"].dt.date]).sum()
    loc_duration = (loc_duration.unstack() / pd.Timedelta("1m") )
    loc_duration["all"] = all_duration / pd.Timedelta("1m")
    return (loc_duration["Other"].fillna(0) / loc_duration["all"] * 0.5 + loc_duration["Work"].fillna(0) / loc_duration["all"]).reset_index(drop=True)
    
    
def compute_percentage_interval(projs, begin_interval='12:30:00', end_interval='13:30:00'):
    type_A = ((projs["End"].dt.tz_localize(None).dt.time >= pd.to_datetime(end_interval).time()) & (projs["Begin"].dt.tz_localize(None).dt.time <= pd.to_datetime(begin_interval).time())) * pd.Timedelta('1h') 
    type_B = ((projs["Begin"].dt.tz_localize(None).dt.time >= pd.to_datetime(begin_interval).time()) & (projs["End"].dt.tz_localize(None).dt.time <= pd.to_datetime(end_interval).time())) * projs["Duration"]
    type_C = ((projs["Begin"].dt.tz_localize(None).dt.time < pd.to_datetime(begin_interval).time()) & (projs["End"].dt.tz_localize(None).dt.time >= pd.to_datetime(begin_interval).time()) & (projs["End"].dt.tz_localize(None).dt.time <= pd.to_datetime(end_interval).time())) * (projs["End"].dt.tz_localize(None) - pd.to_datetime(projs["End"].dt.date.astype(str) + " " + begin_interval))
    type_D = ((projs["Begin"].dt.tz_localize(None).dt.time >= pd.to_datetime(begin_interval).time()) & (projs["Begin"].dt.tz_localize(None).dt.time <= pd.to_datetime(end_interval).time()) & (projs["End"].dt.tz_localize(None).dt.time > pd.to_datetime(end_interval).time())) * (pd.to_datetime(projs["Begin"].dt.date.astype(str) + " " + end_interval) - projs["Begin"].dt.tz_localize(None))

    duration_lunch = type_A + type_B + type_C + type_D
    all_duration = projs["Duration"].groupby([projs["Activity"],  projs["Begin"].dt.date]).sum()

    return (duration_lunch.groupby([projs["Activity"], projs["Begin"].dt.date]).sum() / all_duration).reset_index().sort_values(["Activity", "Begin"])[0]

def compute_factor_lunch_work(projs, begin_lunch='12:00:00', end_lunch='13:00:00'):
    return compute_percentage_interval(projs, begin_lunch, end_lunch)

def compute_factor_evening_work(projs, begin_evening='17:30:00', end_evening='23:59:59'):
    return compute_percentage_interval(projs, begin_evening, end_evening)


def compute_factor_calendar_activities(pdaily, calendar):
    caldf = pd.DataFrame(calendar["Subject"])
    caldf["Start"] = pd.to_datetime(calendar["Start Date"] + " " + calendar["Start Time"], format="%d-%m-%Y %H:%M:%S").dt.tz_localize("Europe/Amsterdam")
    caldf["End"] = pd.to_datetime(calendar["End Date"] + " " + calendar["End Time"], format="%d-%m-%Y %H:%M:%S").dt.tz_localize("Europe/Amsterdam")
    events_day = caldf.groupby(caldf["Start"].dt.date)["Subject"].count()
    events_day.index = pd.to_datetime(events_day.index).tz_localize("Europe/Amsterdam")
    return pd.merge(pdaily, events_day, how="left", left_on="Begin", right_index=True)["Subject"].fillna(0)    


In [ ]:
pd.concat([pdaily["Begin"], pdaily["Begin"].dt.dayofweek, (pdaily["Begin"].dt.dayofweek > 5) * 1.0],axis=1)

In [ ]:
# RCIS
# 10 Feb 2023 submission
# 31 Mar 2023 camera ready
# 25 May 2023 presentation
# 1 Oct 2023 extension


# ICPM
# 13 Jun 2023 (original deadline)
# 20 Jun 2023 submission
# 5 sep 2023 Camera ready
# 24 oct 2023 presentation

deadlines = {
    "RCIS paper": [
        pd.Timestamp(year=2023, month=2, day=11, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=4, day=1, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=5, day=25, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=10, day=1, tz="Europe/Amsterdam"),
    ], 

    "ICPM on AWT": [
        pd.Timestamp(year=2023, month=6, day=13, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=6, day=21, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=9, day=6, tz="Europe/Amsterdam"),
        pd.Timestamp(year=2023, month=10, day=24, tz="Europe/Amsterdam")
    ],

    "Grand challenges": [],

    "Map metaphor": []
}

factors = pd.concat([
    pdaily["Begin"], pdaily["Activity"], 
    compute_factor_deadline(pdaily, deadlines).rename("ClosenessToDeadline"), 
    compute_factor_others(pdaily, deadlines).rename("ClosenessToDeadlineOthers"), 
    compute_factor_calendar_activities(pdaily, calendar_csv).rename("ScheduledActivities"), 
    compute_factor_trip(pdaily, trip_dates).rename("ClosenessToTrip"),
    compute_location(projs, location_mapped).rename("VicinityOfOthers"),
    compute_factor_weekend_work(pdaily).rename("WeekendWork"),
    compute_factor_evening_work(projs).rename("EveningWork"),
    compute_factor_lunch_work(projs).rename("LunchWork"), 
    compute_factor_summer_work(pdaily).rename("SummerWork")],
    axis=1)

factors

# Correlation

It computes pearson correlation between metrics and factors

In [ ]:
all_merged = pd.merge(daily_metrics, factors, on=["Activity", "Begin"])
all_merged.head(10)

In [ ]:
from scipy.stats import pearsonr
all_merged.drop(["Activity", "Begin"], axis=1).corr(method=lambda x, y: pearsonr(x, y)[0]).drop(index=np.delete(daily_metrics.columns, np.where(daily_metrics.columns=='ActivityVariety')), columns=np.append(factors.columns.values, 'ActivityVariety') , errors='ignore')

In [ ]:
print("P-values < 0.01")
(all_merged.drop(["Activity", "Begin"], axis=1).corr(method=lambda x, y: pearsonr(x, y)[1]) < 0.05).drop(index=np.delete(daily_metrics.columns, np.where(daily_metrics.columns=='ActivityVariety')), columns=np.append(factors.columns.values, 'ActivityVariety') , errors='ignore')